# Lesson 6: Planning and Stock Report Generation

## Setup

In [49]:
import os
from dotenv import load_dotenv

962389a037aa4cbda606fe503fe42400


In [56]:
config_list = [
  
    {
        'model': 'gpt4',
        'api_key': ''+(os.environ.get('AZURE_OPENAI_KEY'))+'',
        'base_url':  ''+(os.environ.get('AZURE_OPENAI_ENDPOINT'))+'',
        'api_type': 'azure',
        'api_version': '2024-02-01'
    }
]


## The task!

In [57]:
task = "Write a blogpost about the stock price performance of "\
"MSFT in the past month. Today's date is 2024-05-24."


## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [58]:
import autogen

In [59]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config={
        "config_list": config_list,
    },
    human_input_mode="ALWAYS",
)

In [60]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config={
        "config_list": config_list,
    }
)

In [61]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config={
        "config_list": config_list,
    },
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

**Note**: In this lesson, you'll use an alternative method of code execution by providing a dict config. However, you can always use the LocalCommandLineCodeExecutor if you prefer. For more details about code_execution_config, check this: https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent/#__init__

In [62]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [63]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config={
        "config_list": config_list,
    },
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

## Define the group chat

In [64]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [65]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config={
        "config_list": config_list,
    }
)


## Start the group chat!

<p style="background-color:#ECECEC; padding:15px; "> <b>Note:</b> In this lesson, you will use GPT 4 for better results. Please note that the lesson has a quota limit. If you want to explore the code in this lesson further, we recommend trying it locally with your own API key.

In [66]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of MSFT in the past month. Today's date is 2024-05-24.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

1. The daily closing prices of Microsoft (MSFT) stocks for the past month. 

 - To plot a graph showing the performance of the stock price.
 - To analyze the trend.

2. Data on any significant events/announcements made by Microsoft in the past month.

 - To correlate the changes in stock prices with these events.
 - To provide a complete analysis.

3. Comparison data of MSFT's performance with industry or market indices (e.g., S&P 500) over the same time period.

 - To compare the performance of MSFT against the overall market.

4. Analysis tools or packages to process the collected data.

 - To perform data cleaning, analysis, and visualization.

Check each task after its completion, provide insights on the progress, 

## Add a speaker selection policy

In [67]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config={
        "config_list": config_list,
    },
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config={
        "config_list": config_list,
    }
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config={
        "config_list": config_list,
    },
    description="Write code based on the plan "
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config={
        "config_list": config_list,
    },
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [68]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [69]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config={
        "config_list": config_list,
    }
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of MSFT in the past month. Today's date is 2024-05-24.

--------------------------------------------------------------------------------



>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

1. The daily closing prices of Microsoft (MSFT) stocks for the past month. 

 - To plot a graph showing the performance of the stock price.
 - To analyze the trend.

2. Data on any significant events/announcements made by Microsoft in the past month.

 - To correlate the changes in stock prices with these events.
 - To provide a complete analysis.

3. Comparison data of MSFT's performance with industry or market indices (e.g., S&P 500) over the same time period.

 - To compare the performance of MSFT against the overall market.

4. Analysis tools or packages to process the collected data.

 - To perform data cleaning, analysis, and visualization.

Check each task after its completion, provide insights on the progress, and guide through the remaining tasks. If a task fails, trying alternative methods or sources to obtain the required data. For example, if a planned data source fails, you may need to find alternate financial data 

**Note**: You might experience slightly different interactions between the agents. The engineer agent may write incorrect code, which the executor agent will report and send back for correction. This process could go through multiple rounds.